### DMV Analysis and predictions

In [68]:
import pandas as pd
from os.path import exists # check if SQL exists
import re
import smtplib, ssl
import time
from numpy import random # random wait time between agencies
from datetime import datetime
from datetime import timedelta
import random #for random refresh times
from pprint import pprint
import pickle


In [101]:
import ipywidgets as widgets

## Intro

## Manipulation steps

## Model

In [111]:
w = widgets.Dropdown(
    options=['Bakers_Basin', 'Bayonne', 'Camden', 'Cherry_Hill', 'Cardiff'],
    value='Bakers_Basin',
    description='Number:',
    disabled=False,
)
display(w)

Dropdown(description='Number:', options=('Bakers_Basin', 'Bayonne', 'Camden', 'Cherry_Hill', 'Cardiff'), value…

Loads models

In [112]:
w.value

'Bayonne'

In [11]:
Bayonne = pickle.load(open('Bayonne_model', 'rb'))

Gets time, transforms into df, applies model

In [45]:
now = datetime.today()
#now.weekday() #weekday is also available
now

datetime.datetime(2022, 5, 25, 20, 36, 26, 146832)

In [41]:
df_predict = pd.DataFrame({'days_cat': [now.day], 'hours_cat':[now.hour], 'minutes_cat':[now.minute]})
df_predict

,days_cat,hours_cat,minutes_cat
0,25,20,32


In [50]:
[results] = Bayonne.predict(df_predict) / 3.6e+12 #nano seconds to hours

In [95]:
predicted_time = now + timedelta(hours = results)
print('The best time to try to schedule an appointment is: ' + predicted_time.strftime('%x, %I:%m %p'))

The best time to try to schedule an appointment is: 05/26/22, 02:05 AM
